<a href="https://colab.research.google.com/github/simon-raut/MOVIES_RECOMMENDOR/blob/main/movie_recommendor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q keras-rs

In [2]:
import os
import keras
import tensorflow as tf
import tensorflow_datasets as tfds
import keras_rs

In [3]:
ratings = tfds.load("movielens/100k-ratings", split="train")
movies = tfds.load("movielens/100k-movies", split="train")

In [4]:
for data in ratings.take(1).as_numpy_iterator():
  print(str(data).replace(", '", ",\n '"))

{'bucketized_user_age': np.float32(45.0),
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': np.float32(46.0),
 'timestamp': np.int64(879024327),
 'user_gender': np.True_,
 'user_id': b'138',
 'user_occupation_label': np.int64(4),
 'user_occupation_text': b'doctor',
 'user_rating': np.float32(4.0),
 'user_zip_code': b'53211'}


In [5]:
users_count=(
    ratings.map(lambda x: tf.strings.to_number(x["user_id"], out_type=tf.int32))
    .reduce(tf.constant(0, tf.int32), tf.maximum)
    .numpy()
)

In [6]:
for data in movies.take(1).as_numpy_iterator():
  print(str(data).replace(", '", ",\n '"))

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


In [7]:
movies_count = movies.cardinality().numpy()

In [8]:
def preprocess_rating(x):
  return(
      tf.strings.to_number(x["user_id"], out_type=tf.int32),
      {
          "movie_id": tf.strings.to_number(x["movie_id"], out_type=tf.int32),
          "rating": (x["user_rating"] - 1.0) / 4.0,

      },
  )

In [9]:
shuffled_ratings = ratings.map(preprocess_rating).shuffle(
    100_000, seed=42, reshuffle_each_iteration=False
)
train_ratings = shuffled_ratings.take(80_000).batch(1000).cache()
test_ratings= shuffled_ratings.skip(80_000).take(20_000).batch(1000).cache()

In [16]:
class RetrievalModel(keras.Model):
    def __init__(
        self,
        num_users,
        num_candidates,
        embedding_dimension=32,
        **kwargs,
    ):
        super().__init__(**kwargs)

        self.user_embedding = keras.layers.Embedding(num_users, embedding_dimension)

        self.candidate_embedding = keras.layers.Embedding(
            num_candidates, embedding_dimension
        )

        self.retrieval = keras_rs.layers.BruteForceRetrieval(k=10, return_scores=False)
        self.loss_fn = keras.losses.MeanSquaredError()

    def build(self, input_shape):
        self.user_embedding.build(input_shape)
        self.candidate_embedding.build(input_shape)

        self.retrieval.candidate_embeddings = self.candidate_embedding.embeddings
        self.retrieval.build(input_shape)
        super().build(input_shape)

    def call(self, inputs, training=False):
        user_embeddings = self.user_embedding(inputs)
        result = {
            "user_embeddings": user_embeddings,
        }
        if not training:
            result["predictions"] = self.retrieval(user_embeddings)
        return result

    def compute_loss(self, x, y, y_pred, sample_weight, training=True):
        candidate_id, rating = y["movie_id"], y["rating"]
        user_embeddings = y_pred["user_embeddings"]
        candidate_embeddings = self.candidate_embedding(candidate_id)

        labels = keras.ops.expand_dims(rating, -1)

        scores = keras.ops.sum(
            keras.ops.multiply(user_embeddings, candidate_embeddings),
            axis=1,
            keepdims=True,
        )
        return self.loss_fn(labels, scores, sample_weight)




In [17]:
model = RetrievalModel(users_count +1, movies_count +1)
model.compile(optimizer=keras.optimizers.Adagrad(learning_rate=0.1))

In [18]:
history = model.fit(
    train_ratings, validation_data=test_ratings, validation_freq=5, epochs=100
)

Epoch 1/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.4772
Epoch 2/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4772
Epoch 3/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4771
Epoch 4/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4771
Epoch 5/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step - loss: 0.4771 - val_loss: 0.4836
Epoch 6/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4771
Epoch 7/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4770
Epoch 8/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.4770
Epoch 9/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4770
Epoch 10/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4769 - val_loss: 0.4836
Epoch 11/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4769
Epoch 12/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4768
Epoch 13/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4768
Epoch 14/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4768
Epoch 15/100
80/80 ━━━━━━━━━━━━━

In [19]:
movie_id_to_movie_title = {
    int(x["movie_id"]): x["movie_title"] for x in movies.as_numpy_iterator()
}
movie_id_to_movie_title[0] = ""

In [20]:
user_id = 42
predictions = model.predict(keras.ops.convert_to_tensor([user_id]))
predictions = keras.ops.convert_to_numpy(predictions["predictions"])

print(f"Recommended movies for user {user_id}:")
for movie_id in predictions[0]:
    print(movie_id_to_movie_title[movie_id])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 425ms/step
Recommended movies for user 42:
b'Star Wars (1977)'
b'Raiders of the Lost Ark (1981)'
b'Fargo (1996)'
b'Silence of the Lambs, The (1991)'
b'Godfather, The (1972)'
b'Return of the Jedi (1983)'
b'Empire Strikes Back, The (1980)'
b"Schindler's List (1993)"
b'Pulp Fiction (1994)'
b'Princess Bride, The (1987)'
